    # Objective: To train a neural network to write texts similar to those of Shakespeare.

In [1]:
import random   #need to use it later while developing helper fn 
import numpy as np
import tensorflow as tf  

In [2]:
# directly download the file and start reading and decoding it
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')  #link from official tensorflow tutorial, yay!
text = open(filepath, 'rb').read().decode(encoding='utf-8')  #rb = open file in binary format for reading

text = open(filepath, 'rb').read().decode(encoding='utf-8').lower() #to increase the performance/accuracy of the model,take only lowercase letters(while predicting next char)     

In [3]:
text = text[300000:800000]   #system slow :( So yeah, not gonna train with the whole text
text

" Blunt,\nAnd Rice ap Thomas with a valiant crew;\nAnd many more of noble fame and worth:\nAnd towards London they do bend their course,\nIf by the way they be not fought withal.\n\nDERBY:\nReturn unto thy lord; commend me to him:\nTell him the queen hath heartily consented\nHe shall espouse Elizabeth her daughter.\nThese letters will resolve him of my mind. Farewell.\n\nBUCKINGHAM:\nWill not King Richard let me speak with him?\n\nSheriff:\nNo, my good lord; therefore be patient.\n\nBUCKINGHAM:\nHastings, and Edward's children, Rivers, Grey,\nHoly King Henry, and thy fair son Edward,\nVaughan, and all that have miscarried\nBy underhand corrupted foul injustice,\nIf that your moody discontented souls\nDo through the clouds behold this present hour,\nEven for revenge mock my destruction!\nThis is All-Souls' day, fellows, is it not?\n\nSheriff:\nIt is, my lord.\n\nBUCKINGHAM:\nWhy, then All-Souls' day is my body's doomsday.\nThis is the day that, in King Edward's time,\nI wish't might fal

In [4]:
#convert values to numerical data to help out the system
characters = sorted(set(text))   #That filters out unique char and provides a sorted set
#create 2 dictionaries to convert char into numerical format(index) and vice-versa, store as dic (key,value) pairs
char_to_index = dict((c, i) for i, c in enumerate(characters))    
index_to_char = dict((i, c) for i, c in enumerate(characters))
#char_to_index
#characters

In [5]:
SEQ_LENGTH = 40   #use 40 char to predict the next char
STEP_SIZE = 3     #shift next 3 char 
#i.e,defines how long a sequence shall be and also how many characters to step further to start the next sentence
sentences = []   
next_char = []   

In [6]:
#fill up lists
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):   
    sentences.append(text[i: i + SEQ_LENGTH])   
    next_char.append(text[i + SEQ_LENGTH])

In [7]:
sentences


[' Blunt,\nAnd Rice ap Thomas with a valian',
 'unt,\nAnd Rice ap Thomas with a valiant c',
 ',\nAnd Rice ap Thomas with a valiant crew',
 'nd Rice ap Thomas with a valiant crew;\nA',
 'Rice ap Thomas with a valiant crew;\nAnd ',
 'e ap Thomas with a valiant crew;\nAnd man',
 'p Thomas with a valiant crew;\nAnd many m',
 'homas with a valiant crew;\nAnd many more',
 'as with a valiant crew;\nAnd many more of',
 'with a valiant crew;\nAnd many more of no',
 'h a valiant crew;\nAnd many more of noble',
 ' valiant crew;\nAnd many more of noble fa',
 'liant crew;\nAnd many more of noble fame ',
 'nt crew;\nAnd many more of noble fame and',
 'crew;\nAnd many more of noble fame and wo',
 'w;\nAnd many more of noble fame and worth',
 'And many more of noble fame and worth:\nA',
 ' many more of noble fame and worth:\nAnd ',
 'ny more of noble fame and worth:\nAnd tow',
 'more of noble fame and worth:\nAnd toward',
 'e of noble fame and worth:\nAnd towards L',
 'f noble fame and worth:\nAnd tow

In [8]:
next_char   #provided next correct letter of the above sentences

['t',
 'r',
 ';',
 'n',
 'm',
 'y',
 'o',
 ' ',
 ' ',
 'b',
 ' ',
 'm',
 'a',
 ' ',
 'r',
 ':',
 'n',
 't',
 'a',
 's',
 'o',
 'o',
 't',
 'y',
 'o',
 'e',
 ' ',
 'e',
 ' ',
 'u',
 'e',
 'I',
 'b',
 't',
 ' ',
 'y',
 'h',
 ' ',
 ' ',
 't',
 'o',
 'h',
 'w',
 'h',
 '.',
 'D',
 'B',
 '\n',
 't',
 'n',
 'n',
 ' ',
 'y',
 'o',
 ';',
 'o',
 'e',
 ' ',
 ' ',
 ' ',
 'm',
 'T',
 'l',
 'i',
 't',
 ' ',
 'e',
 ' ',
 't',
 'h',
 'r',
 'l',
 'c',
 's',
 't',
 '\n',
 ' ',
 'a',
 ' ',
 'p',
 's',
 'E',
 'z',
 'e',
 ' ',
 'r',
 'a',
 'h',
 'r',
 'T',
 's',
 'l',
 't',
 's',
 'i',
 ' ',
 's',
 'v',
 'h',
 ' ',
 ' ',
 ' ',
 'n',
 ' ',
 'r',
 'e',
 '.',
 'B',
 'K',
 'G',
 'M',
 'W',
 'l',
 'o',
 'K',
 'g',
 'i',
 'a',
 ' ',
 't',
 'e',
 'p',
 'k',
 'i',
 ' ',
 'm',
 '\n',
 'e',
 'f',
 '\n',
 ',',
 'y',
 'o',
 ' ',
 'r',
 ' ',
 'e',
 'f',
 'e',
 'e',
 'a',
 'e',
 '.',
 'B',
 'K',
 'G',
 'M',
 'H',
 't',
 'g',
 ' ',
 'd',
 'd',
 'r',
 's',
 'h',
 'd',
 'n',
 'R',
 'e',
 ',',
 'r',
 ',',
 'o',
 ' ',
 'n',


In [9]:
#creating nparray of 0s with shape as given below and data type boolean
x = np.zeros((len(sentences), SEQ_LENGTH,   
              len(characters)), dtype=np.bool)
#We have one dimension for the sentences, the positions of the characters within the sentences and to specify which character is at this position.
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)   #target/output data in a similar format

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1
# x: Wherever a character(char) appears in a certain sentence(satz, having no. 'i') at a certain position(t), we will set it to a one (or True) and other values remain 0. 

C:\Users\user\AppData\Local\Temp\ipykernel_3192\3757493289.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)
C:\Users\user\AppData\Local\Temp\ipykernel_3192\3757493289.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)   #target/output data in a similar format


In [10]:
from tensorflow.keras.models import Sequential  
from tensorflow.keras.optimizers import RMSprop  
from tensorflow.keras.layers import Activation, Dense, LSTM   
#Optimizers are classes to tune our model parameters so as to minimize loss
#RMSprop- Root mean squared propagation - optimiztn algorithm to improve speed adn stability of model training
#Why Long Short Term Memory layer?
#Coz it deals with algorithms that try to mimic the human brain the way it operates and to uncover the underlying relationships in the given sequential data
#LSTM has got a "memory space/cell" called 'cell state' where info retains over time.

In [11]:
model = Sequential()  #create Seq model
model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))    # 1st layer is LSTM of 128 neurons                               
model.add(Dense(len(characters)))                                #Followed by dense hidden layer; finish with the softmax fn
model.add(Activation('softmax'))

In [13]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.01))    #learning rate           
model.fit(x, y, batch_size=512, epochs=5)  # Dear model, you should have 512 examples and see the data 5 times. 

Epoch 1/5
326/326 [==============================] - 654s 2s/step - loss: 2.6066
Epoch 2/5
326/326 [==============================] - 656s 2s/step - loss: 2.3677
Epoch 3/5
326/326 [==============================] - 580s 2s/step - loss: 2.2528
Epoch 4/5
326/326 [==============================] - 594s 2s/step - loss: 2.1789
Epoch 5/5
326/326 [==============================] - 663s 2s/step - loss: 2.1224


Lemme now make this model generate texts(predict texts) instead of just char. probabilities.

In [14]:
#Helper fn to sample an index from a probability array/ to pick one sample char with high temp(risky) or low temp(probably right)
#risky=probably wrong but "interesting" char (which is,again,based on prob given by activation fn for the char)
#Thus, this fn just picks 1 specific char
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)  #draw samples from a multinomial distribution
    return np.argmax(probas)   # finds the argument that gives the maximum value from output/target fn

In [16]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)   #Need to start with some char to predict next char,duh!
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence                                        #Updating "generated"...
    for i in range(length):                                      #Need to produce as many char as the length set by the user,hence the for loop
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))   #Same as before, if char occurs, put 1 on the array.
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1         

        predictions = model.predict(x_predictions, verbose=0)[0]    #First index= softmax result(char with highest prob)
        next_index = sample(predictions, temperature)               #feed into sample fn                        
        next_character = index_to_char[next_index]                  #which gives the numerical representn of the next chosen char

        generated += next_character                                #add char to generated text 
        sentence = sentence[1:] + next_character                   #sentence from 1st letter on+next char
    return generated                                               #all done:)

In [17]:
print(generate_text(500, 0.1))  

ns, soon preys upon itself.
This royal the the the the the the the the sore,
The the the to the the the the the the the seat
The the sour and and and and and and the meat
The the sour and and and and and and and and the the theer
The the the the core the the beat of the soust
The the the the the to the to the the the seat,
The the the the the the the the the the the the the the the the the the the the sores,
The with the the the to the the the the seath the seath the the the the the the the the sore the sores,
And and the the the the 


In [18]:
print(generate_text(500, 0.8))  

ES:
There is no truth at all i' the oract and owe,
And rouplens your in, fard you sear then?

KIOG EDKd:
Yof cheithe fothen, incageat,
And garbest onf that beead fore the ford
In lowe thou aoid the ith of I core-de.

CAORTET:
Oit opomenan thaiche wenpinn you Lace wis
For ine sepearse with mean of mach I wall

Bout He lome on and ir alloesore I whast apnost in

HaWRI HARD
HAD: Goras houl male tighe hour sseco dite.

AURENHE:
Go coalouie yout ows doap ore seair,
Hith I wall padist or pave hit lond:
That you kiss brooguthe rishing mane o


In [19]:
print(generate_text(500, 0.5))  

was a poor groom of thy stable, king,
Whom nothe sall to bee of the cond and hence meas,
And apes's our sueal of pareat bout the peater soon,

ARY HARAR:
She shat the dome rowe the sore: the and dome.

ARI GARI IV:
Whit thou hith, the mere the with wete ho sourt:
And the collon the comes.

HARNGEN:
Me ureat and couth me the tore fore not hear,
And me the pien of thou so tor me in thour
Thes cofat thim more stous as seath and soreld
The cond in poor o ware the lant I wellest thou the theer
Ard more that and or mane the fance the perin

